# Homework 1 LLM Zoomcamp
## Felipe González Urrego

### Q1. Running Elastic

In [1]:
!curl localhost:9200

{
  "name" : "1dd2d60ebfb7",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "iXT3xXE3SG6aSCT9G3ZHug",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


**Answer**:  42f05b9372a9a4a470db3b52817899b99a76ee73

### Getting the data

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing the data

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200')

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
from tqdm.auto import tqdm

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

**Answer**: Index

### Q3. Searching

In [8]:
query = 'How do I execute a command in a running docker container?'

In [9]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [10]:
response = es_client.search(index=index_name, body=search_query)

In [11]:
response

ObjectApiResponse({'took': 68, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 843, 'relation': 'eq'}, 'max_score': 83.48925, 'hits': [{'_index': 'course-questions', '_id': 'WHtvwZABQ4do6I-lG0qg', '_score': 83.48925, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'ZntuwZABQ4do6I-l6EjX', '_score': 75.590034, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker c

**Answer**: 84.05 

### Q4. Filtering


In [12]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        "filter":{
            "term": {
                "course": "machine-learning-zoomcamp"
                    }
            
                }
            }
        }
}

In [13]:
response = es_client.search(index=index_name, body=search_query)

In [14]:
response["hits"]["hits"][2]

{'_index': 'course-questions',
 '_id': 'eHtvwZABQ4do6I-lHkrc',
 '_score': 49.60426,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

**Answer**: How do I copy files from a different folder into docker container’s working directory?

### Q5. Building a prompt

In [15]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [16]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [17]:
context_pieces = []

for hit in response['hits']['hits']:
    doc = hit['_source']
    context_piece = context_template.format(**doc)
    context_pieces.append(context_piece)

context = '\n\n'.join(context_pieces)

In [18]:
prompt = prompt_template.format(question=query, context=context)

In [19]:
len(prompt)

1462

**Answer**: 1462

### Q6. Tokens

In [20]:
import tiktoken

In [21]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [22]:
len(encoding.encode(prompt))

322

### Q7 OPENAI Answer

In [23]:
from openai import OpenAI

In [24]:
client = OpenAI()

In [26]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'To execute a command in a running docker container, you can use the `docker exec` command. Here are the steps to follow:\n\n1. First, find the ID of the running container by listing all running containers:\n   ```bash\n   docker ps\n   ```\n\n2. Then, use the container ID to execute a command in the specific container. For example, to start a bash session, you would use the following command:\n   ```bash\n   docker exec -it <container-id> bash\n   ```\n\nReplace `<container-id>` with the actual ID of your running container. This will open an interactive bash shell inside the container.'

### Q8 Costs